In [ ]:
def main(): 
    import json
    import  graph_construction as function
    import pandas as pd
    import os

    #CARICAMENTO FILE CONFIGURAZIONE
    with open("config.json") as f:
        data=json.load(f)

    #Definizione pathway
    path_data_mutational=data["Paths"]["data_mutational"]
    path_clinical_sample=data["Paths"]["data_clinical_sample"]
    path_clinical_patient=data["Paths"]["data_clinical_patient"]
    column_mutation_name=data["Mutation"]["column_mutation_name"]
    gene_interest=data["Mutation"]["gene_of_interest"]
    path_save=data["Paths"]["output_folder"]
    column_gene=data["Mutation"]["column_gene_name"]
    column_hgvsp=data["Mutation"]["column_hgvsp_short"]
    column_hgvsc=data["Mutation"]["column_hgvsc"]
    column_variant_classification=data["Mutation"]["column_variant_classification"]
    vaf=data["vaf"]
    vaf_score=data["vaf_score"]
    column_vaf=data["column_vaf"]

    #caricamento dei dataframes
    data_mutational,data_clinical_sample,data_clinical_patient=function.read_file(path_data_mutational,path_clinical_sample,path_clinical_patient)
    
    #creazione delle mappe pazienti e varianti + creazione del grafo
    if column_mutation_name=="":
        data_mutational=function.adding_category_mutation(data_mutational,column_gene,column_hgvsp,column_variant_classification,column_hgvsc)

    #filtraggio per la vaf se sempresente
    if vaf and column_vaf!="":
        data_mutational=data_mutational[column_vaf]>=vaf_score
    elif vaf and column_vaf=="":
        data_mutational['t_AF']=data_mutational.apply(function.calculated_vaf, axis=1)


    map_patients,map_variants,map_consequence=function.create_maps(data_mutational,data,column_mutation_name)
    graph=function.graph_creation(map_patients,map_variants)

    #plot tipi di mutazione geni di interesse
    if len(gene_interest)!=0:
        function.plot_mutation_gene(map_consequence,gene_interest,path_save)

    #clusterizzazione
    seed=function.selected_seed(graph)
    dendro=function.leiden_clustering(graph,seed)

    #gestione parte grafica del grafo (aggiunta colori + file per cytoscape)
    graph=function.adding_graph_color(graph,dendro)
    if not os.path.exists(path_save):
        os.makedirs(path_save)
    function.write_graph_to_cytoscape(graph,path_save)


    #creazione della mappa cluster e attribuzione del cluster i pazienti e alle varianti
    map_cluster=function.map_cluster_creation(graph,dendro)
    map_patients,map_variants=function.adding_cluster_to_map(map_cluster,map_patients,map_variants)
    graph=function.cluster_noded_attributes(graph,map_patients,map_variants)

    #aggiunta delle informazioni cliniche alla mappa dei pazienti
    if len(data_clinical_sample)!=0:
        map_patients=function.enriched_sample_data(data_clinical_sample,map_cluster,map_patients)
    if len(data_clinical_patient)!=0:
        map_patients=function.enriched_patient_data(data_clinical_patient,map_patients)

    #creazione file in cui riassumere le informazioni nei diversi cluster
    function.summary_info(path_save,map_cluster,map_patients)
    function.numerosity_info(path_save,map_cluster)

    #creazione di una mappa con il numero di mutazioni per ogni gene + creazione di due mappe con i valori assoluti e percentuali di distribuzione delle mutazioni, per ciascun gene,
    #nei diversi cluster
    gene_total_count=function.count_gene(graph)
    map_cluster_gene_abs,map_cluster_gene_percent=function.count_gene_abs_percent(map_cluster,gene_total_count,path_save)
    #salvataggio delle percentuali di distribuzione delle mutazioni dei diversi geni nei cluster
    function.genes_single_cluster(map_cluster,path_save)

    function.couple_centroid_element(dendro,map_cluster,path_save)
    function.centroids_cluster(dendro,path_save)


    #**************SEZIONE GRAFICA CONNESSIONI***************
    #defizione del numero di connessioni per ogni variante all'interno del cluster 
    variant_patient_connection_count=function.variant_conncection_patient(dendro)
    #defizione del numero di connessioni per ogni paziente all'interno del cluster
    patient_variant_connection_count=function.patient_connection_variant(dendro)
    graph=function.add_size_node(graph,variant_patient_connection_count)
    # CREAZIONE DI UN FILE "CONNECTION_VARIANT" IN CUI SONO INDICATE IL NUMERO DI VARIANTI COMUNI TRA I VARI PAZIENTI DI UN CLUSTER
    function.file_connection_variant(map_cluster,map_patients,path_save)
    # CREAZIONE DI UN FILE "CONCCECTION_PATIENT" IN CUI SONO INDICATI IL NUMERO DI PAZIENTI COMUNI TRA LE VARIE VARIANTI DI UN CLUSTER
    function.file_connection_patient(map_cluster,map_variants,path_save)
    #plott delle connessioni lato varianti:
    data_connection=pd.read_csv("./tcga_filtred/output/connection_patient.csv",sep="\t")
    #for i in range(len(map_cluster.keys())):
      # function.plot_distance_comutated_cluster_variants(dendro,i,data_connection,path_save)


if __name__=="__main__":
    main()

CARICAMENTO DATI

In [28]:
import json
import  graph_construction as function
import os

#CARICAMENTO FILE CONFIGURAZIONE
with open("config.json") as f:
    data=json.load(f)

#Definizione pathway
path_data_mutational=data["Paths"]["data_mutational"]
path_clinical_sample=data["Paths"]["data_clinical_sample"]
path_clinical_patient=data["Paths"]["data_clinical_patient"]
column_mutation_name=data["Mutation"]["column_mutation_name"]
gene_interest=data["Mutation"]["gene_of_interest"]
path_save=data["Paths"]["output_folder"]
column_gene=data["Mutation"]["column_gene_name"]
column_hgvsp=data["Mutation"]["column_hgvsp_short"]
column_hgvsc=data["Mutation"]["column_hgvsc"]
column_variant_classification=data["Mutation"]["column_variant_classification"]
sample_name=data["Clinical_data"]["column_sample_name"]
patient_name=data["Clinical_data"]["column_patient_name"]
vaf=data["Mutation"]["vaf"]
vaf_score=data["Mutation"]["vaf_score"]
column_vaf=data["Mutation"]["vaf_column"]

In [29]:
#caricamento dei dataframes
data_mutational,data_clinical_sample,data_clinical_patient=function.read_file(path_data_mutational,path_clinical_sample,path_clinical_patient)

DIZIONARIO PAZIENTI-DIZIONARIO VARIANTI-COSTRUZIONE GRAFO

In [30]:
#creazione delle mappe pazienti e varianti + creazione del grafo
if column_mutation_name=="":
    #print("ciao")
    data_mutational=function.adding_category_mutation(data_mutational,column_gene,column_hgvsp,column_variant_classification,column_hgvsc)

map_patients,map_variants,map_consequence=function.create_maps(data_mutational,data,column_mutation_name)

graph=function.graph_creation(map_patients,map_variants)


HGVSp_Short Variant_Classification None
ciao


In [31]:
#filtraggio della VAF
if vaf and column_vaf!="":
    data_mutational=data_mutational[data_mutational[column_vaf]>=vaf_score]
elif vaf and column_vaf=="":
    data_mutational['t_AF']=data_mutational.apply(function.calculated_vaf, axis=1)
    print(len(data_mutational))
    data_mutational=data_mutational[data_mutational['t_AF']>=vaf_score]
    print(len(data_mutational))

4777
4745


In [32]:
#plot tipi di mutazione geni di interesse
if len(gene_interest)!=0:
    function.plot_mutation_gene(map_consequence,gene_interest)

CLUSTERIZZAZIONE

In [33]:
#clusterizzazione
seed=function.selected_seed(graph)
dendro=function.leiden_clustering(graph,seed)


numero di clusters: 77 Modularità: 0.9429377973769981


In [18]:
#gestione parte grafica del grafo (aggiunta colori + file per cytoscape)
graph=function.adding_graph_color(graph,dendro)
if not os.path.exists(path_save):
    os.makedirs(path_save)
function.write_graph_to_cytoscape(graph,path_save)



DIZIONARIO CLUSTER

In [19]:
#creazione della mappa cluster e attribuzione del cluster i pazienti e alle varianti
map_cluster=function.map_cluster_creation(graph,dendro)

map_patients,map_variants=function.adding_cluster_to_map(map_cluster,map_patients,map_variants)

graph=function.cluster_noded_attributes(graph,map_patients,map_variants)

INFORMAZIONI CLINICHE

In [20]:
#aggiunta delle informazioni cliniche alla mappa dei pazienti
if len(data_clinical_sample)!=0:
    map_patients=function.enriched_sample_data(data_clinical_sample,map_cluster,map_patients,sample_name,patient_name)
if len(data_clinical_patient)!=0:
    map_patients=function.enriched_patient_data(data_clinical_patient,map_patients,patient_name)

FILE DI SINTESI

In [21]:
#creazione file in cui riassumere le informazioni nei diversi cluster
function.summary_info(path_save,map_cluster,map_patients,patient_name)
function.numerosity_info(path_save,map_cluster)

GENI

In [22]:
#creazione di una mappa con il numero di mutazioni per ogni gene + creazione di due mappe con i valori assoluti e percentuali di distribuzione delle mutazioni, per ciascun gene,
#nei diversi cluster
gene_total_count=function.count_gene(graph)
map_cluster_gene_abs,map_cluster_gene_percent=function.count_gene_abs_percent(map_cluster,gene_total_count,path_save)

#salvataggio delle percentuali di distribuzione delle mutazioni dei diversi geni nei cluster
function.genes_single_cluster(map_cluster,path_save)

In [17]:
function.creation_cluster_clinical_data(map_patients,path_save)

,ACINAR_FRACTION,ADDITIONAL_PATHOLOGIC_FINDINGS,AGE,ALCOHOL_CONSUMPTION,BMI,CANCER_TYPE,CANCER_TYPE_DETAILED,CAUSE_OF_DEATH,CLINICAL_STAGING_DISTANT_METASTASIS_CM,FAT_FRACTION,...,STROMAL_FRACTION,TMB_NONSYNONYMOUS,TOBACCO_SMOKING_HISTORY,TUMOR_FOCALITY,TUMOR_NECROSIS,TUMOR_SITE,TUMOR_SIZE_CM,TUMOR_STAGE_PATHOLOGICAL,VITAL_STATUS,cluster
0,0,Other : Microscopic tumor extension: tumor inv...,69,Alcohol consumption equal to or less than 2 dr...,28.36,Pancreatic Cancer,Pancreatic Adenocarcinoma,NaN,cMX,0,...,73,1.433333,Current reformed smoker within past 15 years,Unifocal,Not identified,Head,4.5,Stage IIA,Living,0
1,1,None identified,42,Alcohol consumption history not available,26.93,Pancreatic Cancer,Pancreatic Adenocarcinoma,Pancreatic Carcinoma,cM0,4,...,60,0.633333,Lifelong non-smoker: Less than 100 cigarettes ...,Unifocal,Not identified,Head,3.0,Stage III,Deceased,2
2,0;0;0,Pancreatic intraepithelial neoplasia : Highest...,68,Alcohol consumption equal to or less than 2 dr...,34.28,Pancreatic Cancer,Pancreatic Adenocarcinoma,Pancreatic Carcinoma,cM0,0;6;0,...,75;55;53,0.300000,"Current reformed smoker, more than 15 years",Unifocal,Not identified,Head,2.7,Stage IIB,Deceased,4
3,0,Other : Gastric GIST,69,Alcohol consumption equal to or less than 2 dr...,24.00,Pancreatic Cancer,Pancreatic Adenocarcinoma,Pancreatic Carcinoma,cMX,0,...,55,0.666667,Lifelong non-smoker: Less than 100 cigarettes ...,Unifocal,Not identified,Tail,5.0,Stage IIB,Deceased,5
4,5;10;5,Chronic pancreatitis,62,Alcohol consumption equal to or less than 2 dr...,22.42,Pancreatic Cancer,Pancreatic Adenocarcinoma,NaN,cM0,0;0;0,...,35;30;32,0.166667,Lifelong non-smoker: Less than 100 cigarettes ...,Unifocal,Not identified,Body,2.8,Stage IIB,Living,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,3,None identified,54,Lifelong non-drinker,23.66,Pancreatic Cancer,Pancreatic Adenocarcinoma,Pancreatic Carcinoma,cMX,0,...,62,1.033333,Lifelong non-smoker: Less than 100 cigarettes ...,Unifocal,Not identified,Tail,3.0,Stage IIB,Deceased,76
136,0;0,None identified,42,Lifelong non-drinker,24.38,Pancreatic Cancer,Pancreatic Adenocarcinoma,Pancreatic Carcinoma,cMX,0;0,...,65;32,0.900000,Lifelong non-smoker: Less than 100 cigarettes ...,Unifocal,Not identified,Head,3.5,Stage III,Deceased,19
137,0,Pancreatic intraepithelial neoplasia : Highest...,75,Alcohol consumption equal to or less than 2 dr...,16.00,Pancreatic Cancer,Pancreatic Adenocarcinoma,Pancreatic Carcinoma,cMX,0,...,48,1.100000,Current smoker: Includes daily and non-daily s...,Unifocal,Present,Head,4.0,Stage IV,Deceased,30
138,15,Pancreatic intraepithelial neoplasia : Highest...,66,Lifelong non-drinker,19.00,Pancreatic Cancer,Pancreatic Adenocarcinoma,Complication Due To Medical/Surgical Care,cMX,0,...,30,0.666667,Lifelong non-smoker: Less than 100 cigarettes ...,Multifocal,Present,Head And Body,7.0,Stage III,Deceased,1


In [23]:
 #function.couple_centroid_element(dendro,map_cluster,path_save)
function.centroids_cluster(dendro,path_save)

Mut_TTC22_p.R512C
Mut_FMN2_p.A111T
Mut_LRP1B_p.Q931L
Mut_DES_p.G19S
Mut_SGPP2_p.M391V
Mut_DRD5_p.A363T
Mut_ADAM29_p.R280Q
Mut_SEMA5A_p.R491L
Mut_MARCH11_p.R193Q
Mut_RACK1_p.D48N
Mut_SCUBE3_p.E561K
Mut_MRM2_p.K225E
Mut_WIPF3_p.P434L
Mut_SMO_p.K674N
Mut_TSGA13_p.E187K
Mut_TAF1L_p.R842W
Mut_SLC39A12_p.E431K
Mut_OR51Q1_p.I47M
Mut_PAX6_p.Q398*
Mut_FOLR1_p.P243T
Mut_DIXDC1_p.D12N
Mut_ADAMTS8_p.L21del
Mut_HOXC10_p.N6K
Mut_PTPRQ_p.P226A
Mut_TEP1_p.R535Q
Mut_G2E3_p.L2P
Mut_MDGA2_p.E745Q
Mut_KTN1_p.S1289*
Mut_RYR3_p.R3733H
Mut_HERC1_p.G695V
Mut_TSC2_p.R691C
Mut_RBFOX1_p.A82T
Mut_TP53_p.Y234_G244delinsC
Mut_PSG9_p.R313Q
Mut_IGLON5_p.R209H
Mut_KIR2DL3_p.R314K
Mut_TRIM28_p.V478A
Mut_NINL_p.R754H
Mut_SPECC1L_p.S120T
Mut_RTCB_p.G489K
Mut_FAM9A_p.Q160Tfs*8
Mut_HTR2C_p.T271M
C3L-00017
Mut_KRAS_p.G12D
Mut_SMAD4_p.C71W
Mut_ADGRE2_p.L160V
Mut_GSK3A_p.L228P
Mut_CSMD2_p.C3138F
Mut_TMEM206_p.S338Kfs*10
Mut_EIF5B_p.S157*
Mut_TGFBR2_p.V454D
Mut_EPHA5_p.M932I
Mut_PHKG1_p.R373Q
Mut_PRSS1_p.G136S
Mut_SLC18A1_p.F8

In [24]:
#defizione del numero di connessioni per ogni variante all'interno del cluster 
variant_patient_connection_count=function.variant_conncection_patient(dendro)
#defizione del numero di connessioni per ogni paziente all'interno del cluster
patient_variant_connection_count=function.patient_connection_variant(dendro)

In [25]:
graph=function.add_size_node(graph,variant_patient_connection_count)

In [26]:
function.plot_graph(graph,path_save,"all")

In [27]:
for v in map_patients["C3N-01715"]["variants"]:
    print(v,map_variants[v]["patients"])

Mut_SLAMF1_p.S277Afs*54 {'C3N-01715'}
Mut_GLTSCR1_p.P937Hfs*10 {'C3N-01715'}
Mut_ADCY8_p.K765R {'C3N-01715'}
Mut_ALPK1_p.T502A {'C3N-01715'}
Mut_SRGAP1_p.A814Pfs*20 {'C3N-01715'}
Mut_NAA25_p.K939Dfs*26 {'C3N-01715'}
Mut_TTN_p.K24346R {'C3N-01715'}
Mut_PLEKHA4_p.G127C {'C3N-01715'}
Mut_OTOP3_p.L594M {'C3N-01715'}
Mut_ADCYAP1R1_p.R185H {'C3N-01715'}
Mut_SESN2_p.P335Lfs*17 {'C3N-01715'}
Mut_NR5A2_p.Y489C {'C3N-01715'}
Mut_AHSA1_p.F80del {'C3N-01715'}
Mut_TECPR1_p.A933T {'C3N-01715'}
Mut_PACS2_p.S97del {'C3N-01715'}
Mut_ABO_p.S272L {'C3N-01715'}
Mut_CACNG3_p.S172Kfs*31 {'C3N-01715'}
Mut_FSIP2_p.M299del {'C3N-01715'}
Mut_ANK2_p.A1809V {'C3N-01715'}
Mut_VGLL1_p.P127S {'C3N-01715'}
Mut_VPS13A_p.A1358V {'C3N-01715'}
Mut_NR5A1_p.H317R {'C3N-01715'}
Mut_PARP1_p.E842G {'C3N-01715'}
Mut_RUVBL1_p.S37A {'C3N-01715'}
Mut_C10orf35_p.R49Pfs*18 {'C3N-01715'}
Mut_CAPN1_p.L126I {'C3N-01715'}
Mut_EFS_p.P215T {'C3N-01715'}
Mut_RPL22_p.K15Rfs*5 {'C3N-01715'}
Mut_MGAT2_p.A393V {'C3N-01715'}
Mut_YARS_p.V407I {

In [49]:
# CREAZIONE DI UN FILE "CONNECTION_VARIANT" IN CUI SONO INDICATE IL NUMERO DI VARIANTI COMUNI TRA I VARI PAZIENTI DI UN CLUSTER
function.file_connection_variant(map_cluster,map_patients,path_save)

# CREAZIONE DI UN FILE "CONCCECTION_PATIENT" IN CUI SONO INDICATI IL NUMERO DI PAZIENTI COMUNI TRA LE VARIE VARIANTI DI UN CLUSTER
function.file_connection_patient(map_cluster,map_variants,path_save)

In [ ]:
import pandas as pd
data_connection=pd.read_csv("./pdac/output/connection_patient.csv",sep="\t")

for i in range(len(map_cluster.keys())):
    function.plot_distance_comutated_cluster_variants(dendro,i,data_connection,path_save)